In [8]:
from google.cloud import bigquery

In [9]:
PROJECT_ID   = "bq-ai-hackaton"
LOCATION     = "us-central1"
GEMINI_MODEL = "gemini-2.5-pro"

In [10]:
client = bigquery.Client(project=PROJECT_ID, location=LOCATION)

In [4]:
VIDEO_ANALYSIS_PROMPT = """
# ROLE AND GOAL
You are a state-of-the-art AI visual analysis system with an expert specialization in human identification within video footage.
Your primary mission is to analyze the provided video for a critical missing person case with the highest degree of accuracy and diligence.
You must be methodical and detail-oriented in your analysis and reporting.

# TASK CONTEXT
This is a high-priority, time-sensitive analysis.
The provided video is a security footage from the street.
The objective is to determine if the missing person is visible in this video, and if so, to extract all relevant information about their presence.

# MISSING PERSON DATA
Carefully analyze the following description of the missing person. Every detail is crucial.

- **Gender:** `{gender}`
- **Approximate Age:** `{age}`
- **Build / Height:** `{build_height}`
- **Hair Color and Style:** `{hair}`
- **Clothing (Top):** `{clothing_top}`
- **Clothing (Bottom):** `{clothing_botton}`
- **Footwear:** `{footwear}`
- **Accessories:** `{accessories}`
- **Distinguishing Features:** `{features}`

# ANALYSIS INSTRUCTIONS
You must perform the following steps in your analysis:

1.  **Full Video Scan:** Meticulously review the entire video from start to finish. Do not stop after a potential first match; the person may appear multiple times.
2.  **Feature Matching:** Compare every individual in the video against the `MISSING PERSON DATA`. Assess matches based on all available criteria: clothing, build, hair, accessories, and any visible distinguishing features or mannerisms.
3.  **Confidence Score Calculation:** Based on your feature matching, calculate a confidence score from `0.0` (no match) to `1.0` (positive identification with multiple strong corroborating features). The score must be based on the quality and quantity of matching attributes. A partial clothing match is a low score; a match on clothing, build, and a distinguishing feature is a very high score.
4.  **Justification:** You MUST provide a step-by-step justification for your confidence score. List the features that matched, the features that did not match, and any features that were ambiguous or obscured (e.g., 'Face was unclear, but clothing is a 90% match').
5.  **Contextual Analysis (If Found):** If you identify the person with a confidence score greater than `0.7`:
    -   Note the exact timestamp(s) (in `HH:MM:SS` format) of their appearance.
    -   Describe their actions and behavior (e.g., 'walking quickly', 'talking on the phone', 'sitting on a bench', 'seemed distressed').
    -   Analyze if they are with anyone else. If so, provide a detailed description of each companion (gender, estimated age, clothing, etc.).
    -   Describe their direction of travel.
"""
VIDEO_ANALYSIS_PROMPT_OUTPUT = """
# OUTPUT FORMAT
Your final output MUST be a single JSON object. Do not include any text or explanations outside of this JSON structure.

**If the person is found (confidence > 0.7):**
```json
{
  "personFound": true,
  "confidenceScore": <float between 0.7 and 1.0>,
  "matchJustification": "A detailed explanation of why the confidence score was given. List all matching and non-matching features. Example: 'Confidence is high due to a perfect match on the red hooded sweatshirt, blue jeans, and black backpack. Subject's build and hair color are also consistent. Face was partially obscured, preventing a 1.0 score.'",
  "summaryOfFindings": "A concise, human-readable summary of the events. Example: 'The missing person was spotted at 00:14:32 walking east on the station platform. They appeared to be alone and were walking at a normal pace while looking at their phone.'",
  "appearances": [
    {
      "timestampStart": "HH:MM:SS",
      "timestampEnd": "HH:MM:SS",
      "actionsAndBehavior": "Detailed description of what the subject is doing during this specific timeframe.",
      "directionOfTravel": "e.g., Northbound, Towards the exit, Away from the camera",
      "companions": [
        {
          "description": "Detailed description of companion 1: Gender, estimated age, build, hair, clothing (top, bottom, shoes), and any notable accessories or features."
        }
      ]
    }
  ]
}
```
**If the person is found (confidence > 0.7):**
```json
{
  "personFound": false,
  "confidenceScore": <float between 0.0 and 0.7>,
  "matchJustification": "Explain why the person was not found or why confidence is low. Example: 'Scanned the entire video. Several individuals matched the general build, but none matched the specific red hooded sweatshirt and white sneaker combination described. Therefore, confidence is very low.'",
  "summaryOfFindings": "A concise statement of the negative finding. Example: 'After a thorough analysis of the entire video footage, there was no individual matching the provided description of the missing person.'",
  "appearances": [],
  "companions": []
}
```
"""

In [5]:
VIDEO_ANALYSIS_PROMPT = VIDEO_ANALYSIS_PROMPT.format(
gender            ="Female",
age               ="35",
build_height      ="1.7m, 80kg",
hair              ="Black hair, sporty",
clothing_top      ="Blue shirt",
clothing_botton   ="Blue pants",
footwear          ="White snikers",
accessories       ="Dark glasses",
features          ="None"
)


In [6]:
VIDEO_ANALYSIS_PROMPT = VIDEO_ANALYSIS_PROMPT + "\n" +VIDEO_ANALYSIS_PROMPT_OUTPUT

In [11]:
VIDEO_ANALYSIS_BQ_QUERY = """
SELECT
  uri,
  AI.GENERATE(("{VIDEO_ANALYSIS_PROMPT}", '\nRecording:  ', OBJ.GET_ACCESS_URL(ref, 'r')),
    connection_id => 'bq-ai-hackaton.us-central1.homeward_gcp_connection',
    endpoint => 'gemini-2.5-pro',
    model_params => JSON '{"generation_config": {"temperature": 0}}').result
FROM `homeward.video_objects`;
""".replace("{VIDEO_ANALYSIS_PROMPT}",VIDEO_ANALYSIS_PROMPT.encode('unicode-escape').replace(b'"', b'\\"').decode("utf-8"))


In [12]:
VIDEO_ANALYSIS_BQ_QUERY

'\nSELECT\n  uri,\n  AI.GENERATE(("\\n# ROLE AND GOAL\\nYou are a state-of-the-art AI visual analysis system with an expert specialization in human identification within video footage.\\nYour primary mission is to analyze the provided video for a critical missing person case with the highest degree of accuracy and diligence.\\nYou must be methodical and detail-oriented in your analysis and reporting.\\n\\n# TASK CONTEXT\\nThis is a high-priority, time-sensitive analysis.\\nThe provided video is a security footage from the street.\\nThe objective is to determine if the missing person is visible in this video, and if so, to extract all relevant information about their presence.\\n\\n# MISSING PERSON DATA\\nCarefully analyze the following description of the missing person. Every detail is crucial.\\n\\n- **Gender:** `Female`\\n- **Approximate Age:** `35`\\n- **Build / Height:** `1.7m, 80kg`\\n- **Hair Color and Style:** `Black hair, sporty`\\n- **Clothing (Top):** `Blue shirt`\\n- **Cloth

In [13]:
# Execute the video analysis query
try:
    # Run the query
    query_job = client.query(VIDEO_ANALYSIS_BQ_QUERY)
    results = query_job.result()

    print("Query completed successfully!")
    print(f"Total rows returned: {results.total_rows}")
    print("\nResults:")

    # Process and display results
    for i, row in enumerate(results, 1):
        print(f"\nVideo {i}:")
        print(f"URI: {row.uri}")
        print("Analysis Result:")
        print(row.result)
        print("-" * 30)

except Exception as e:
    print(f"Error executing query: {str(e)}")
    print(f"Error type: {type(e).__name__}")

Error executing query: 400 Syntax error: Unclosed string literal at [4:4980]; reason: invalidQuery, location: query, message: Syntax error: Unclosed string literal at [4:4980]

Location: us-central1
Job ID: 92f15b18-87b2-4ca5-9b4c-3c088d1176c1

Error type: BadRequest


In [20]:
# SQL Query Constants
MISSING_PERSON_INSERT_QUERY = """
MERGE `homeward.missing_persons` AS target
USING (
  SELECT
    @id AS id,
    @case_number AS case_number,
    @name AS name,
    @surname AS surname,
    @date_of_birth AS date_of_birth,
    @gender AS gender,
    @height AS height,
    @weight AS weight,
    @hair_color AS hair_color,
    @eye_color AS eye_color,
    @distinguishing_marks AS distinguishing_marks,
    @clothing_description AS clothing_description,
    @last_seen_date AS last_seen_date,
    @last_seen_time AS last_seen_time,
    @last_seen_address AS last_seen_address,
    @last_seen_city AS last_seen_city,
    @last_seen_country AS last_seen_country,
    @last_seen_postal_code AS last_seen_postal_code,
    @last_seen_latitude AS last_seen_latitude,
    @last_seen_longitude AS last_seen_longitude,
    CASE
      WHEN @last_seen_latitude IS NOT NULL AND @last_seen_longitude IS NOT NULL
      THEN ST_GEOGPOINT(@last_seen_longitude, @last_seen_latitude)
      ELSE NULL
    END AS last_seen_geo,
    @circumstances AS circumstances,
    @priority AS priority,
    @status AS status,
    @description AS description,
    @medical_conditions AS medical_conditions,
    @additional_info AS additional_info,
    @photo_url AS photo_url,
    @reporter_name AS reporter_name,
    @reporter_phone AS reporter_phone,
    @reporter_email AS reporter_email,
    @relationship AS relationship,
    CURRENT_TIMESTAMP() AS created_date,
    CURRENT_TIMESTAMP() AS updated_date,
    AI.GENERATE(
      CONCAT(
        'Generate a comprehensive summary paragraph for this missing person case for law enforcement analysis and matching purposes. ',
        'Write it as a single, flowing, discursive paragraph without bullet points, lists, or structured formatting. ',
        'Include key identifying features, circumstances, and critical search information in narrative form. ',
        'Return only the summary paragraph without any introduction, conclusion, or additional commentary from the model. ',
        'Person: ', @name, ' ', @surname, ', ',
        'Age: ', CAST(DATE_DIFF(CURRENT_DATE(), @date_of_birth, YEAR) AS STRING), ' years old, ',
        'Gender: ', @gender, ', ',
        CASE
          WHEN @height IS NOT NULL THEN CONCAT('Height: ', CAST(@height AS STRING), 'cm, ')
          ELSE ''
        END,
        CASE
          WHEN @weight IS NOT NULL THEN CONCAT('Weight: ', CAST(@weight AS STRING), 'kg, ')
          ELSE ''
        END,
        CASE
          WHEN @hair_color IS NOT NULL THEN CONCAT('Hair: ', @hair_color, ', ')
          ELSE ''
        END,
        CASE
          WHEN @eye_color IS NOT NULL THEN CONCAT('Eyes: ', @eye_color, ', ')
          ELSE ''
        END,
        CASE
          WHEN @distinguishing_marks IS NOT NULL THEN CONCAT('Distinguishing marks: ', @distinguishing_marks, '. ')
          ELSE ''
        END,
        CASE
          WHEN @clothing_description IS NOT NULL THEN CONCAT('Last seen wearing: ', @clothing_description, '. ')
          ELSE ''
        END,
        'Last seen on ', CAST(@last_seen_date AS STRING),
        CASE
          WHEN @last_seen_time IS NOT NULL THEN CONCAT(' at ', CAST(@last_seen_time AS STRING))
          ELSE ''
        END,
        ' in ', @last_seen_city, ', ', @last_seen_country, '. ',
        'Location: ', @last_seen_address,
        CASE
          WHEN @last_seen_postal_code IS NOT NULL THEN CONCAT(', ', @last_seen_postal_code)
          ELSE ''
        END,
        '. Circumstances: ', @circumstances, '. ',
        CASE
          WHEN @medical_conditions IS NOT NULL THEN CONCAT('Medical conditions: ', @medical_conditions, '. ')
          ELSE ''
        END,
        CASE
          WHEN @additional_info IS NOT NULL THEN CONCAT('Additional information: ', @additional_info, '. ')
          ELSE ''
        END,
        'Case priority: ', @priority, ', Status: ', @status, '. ',
        'Reported by: ', @reporter_name, ' (', @relationship, ').'
      ),
      connection_id => 'bq-ai-hackaton.us-central1.homeward_gcp_connection',
      endpoint => 'gemini-2.5-flash',
      model_params => JSON '{"generation_config": {"temperature": 0}}'
    ).result AS ml_summary
) AS source
ON target.id = source.id
WHEN NOT MATCHED THEN
  INSERT (
    id, case_number, name, surname, date_of_birth, gender,
    height, weight, hair_color, eye_color, distinguishing_marks, clothing_description,
    last_seen_date, last_seen_time, last_seen_address, last_seen_city, last_seen_country,
    last_seen_postal_code, last_seen_latitude, last_seen_longitude, last_seen_geo,
    circumstances, priority, status, description, medical_conditions, additional_info,
    photo_url, reporter_name, reporter_phone, reporter_email, relationship,
    created_date, updated_date, ml_summary
  )
  VALUES (
    source.id, source.case_number, source.name, source.surname, source.date_of_birth, source.gender,
    source.height, source.weight, source.hair_color, source.eye_color, source.distinguishing_marks, source.clothing_description,
    source.last_seen_date, source.last_seen_time, source.last_seen_address, source.last_seen_city, source.last_seen_country,
    source.last_seen_postal_code, source.last_seen_latitude, source.last_seen_longitude, source.last_seen_geo,
    source.circumstances, source.priority, source.status, source.description, source.medical_conditions, source.additional_info,
    source.photo_url, source.reporter_name, source.reporter_phone, source.reporter_email, source.relationship,
    source.created_date, source.updated_date, source.ml_summary
  );
"""

VERIFY_RECORD_QUERY = """
SELECT
    id,
    case_number,
    name,
    surname,
    date_of_birth,
    gender,
    height,
    weight,
    hair_color,
    eye_color,
    distinguishing_marks,
    clothing_description,
    last_seen_date,
    last_seen_time,
    last_seen_city,
    circumstances,
    priority,
    status,
    medical_conditions,
    additional_info,
    reporter_name,
    relationship,
    created_date,
    ml_summary
FROM `homeward.missing_persons`
WHERE id = @case_id
ORDER BY created_date DESC
LIMIT 1;
"""

In [21]:
# Refactored functions using constants
import uuid
from datetime import date, time


def create_missing_person_insert_query():
    """
    Get the parameterized SQL query for inserting missing person with ML summary.
    Uses BigQuery parameters for safe and efficient execution.

    Returns:
        str: The SQL query string with parameters
    """
    return MISSING_PERSON_INSERT_QUERY


def get_verify_record_query():
    """
    Get the parameterized SQL query for verifying inserted records.

    Returns:
        str: The SQL query string with parameters
    """
    return VERIFY_RECORD_QUERY


# Sample data for testing - this will be passed as parameters
sample_missing_person = {
    "id": str(uuid.uuid4()),
    "case_number": "CASE-2024-034",
    "name": "John",
    "surname": "Doe",
    "date_of_birth": date(1990, 1, 15),
    "gender": "Male",
    "height": 175.0,
    "weight": 70.0,
    "hair_color": "Brown",
    "eye_color": "Green",
    "distinguishing_marks": "Tattoo on right arm",
    "clothing_description": "Black jacket, blue jeans, white sneakers",
    "last_seen_date": date(2024, 8, 20),
    "last_seen_time": time(18, 45, 0),
    "last_seen_address": "456 Oak Avenue",
    "last_seen_city": "San Francisco",
    "last_seen_country": "USA",
    "last_seen_postal_code": "94103",
    "last_seen_latitude": 37.7849,
    "last_seen_longitude": -122.4094,
    "circumstances": "Failed to return home after evening jog",
    "priority": "High",
    "status": "Active",
    "description": "Missing marathon runner",
    "medical_conditions": "Diabetes - requires medication",
    "additional_info": "Regular jogger, knows the area well",
    "photo_url": "https://example.com/photos/john_doe.jpg",
    "reporter_name": "Jane Doe",
    "reporter_phone": "4155559876",
    "reporter_email": "jane.doe@email.com",
    "relationship": "Wife"
}

INSERT_MISSING_PERSON_QUERY = create_missing_person_insert_query()

print("✅ Refactored functions using SQL constants!")
print(f"📋 Sample case ID: {sample_missing_person['id']}")
print(f"📋 Sample case number: {sample_missing_person['case_number']}")
print(f"👤 Sample person: {sample_missing_person['name']} {sample_missing_person['surname']}")
print("\nQuery uses BigQuery parameters (@parameter_name) for safe execution.")

✅ Refactored functions using SQL constants!
📋 Sample case ID: a54701be-c15e-47a7-ae1b-3c14c1c5b040
📋 Sample case number: CASE-2024-034
👤 Sample person: John Doe

Query uses BigQuery parameters (@parameter_name) for safe execution.


In [22]:
# Execute the parameterized missing person insert query with ML summary generation
from google.cloud import bigquery

try:
    print("Executing parameterized missing person insert with ML summary generation...")

    # Configure the query job with parameters
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("id", "STRING", sample_missing_person["id"]),
            bigquery.ScalarQueryParameter("case_number", "STRING", sample_missing_person["case_number"]),
            bigquery.ScalarQueryParameter("name", "STRING", sample_missing_person["name"]),
            bigquery.ScalarQueryParameter("surname", "STRING", sample_missing_person["surname"]),
            bigquery.ScalarQueryParameter("date_of_birth", "DATE", sample_missing_person["date_of_birth"]),
            bigquery.ScalarQueryParameter("gender", "STRING", sample_missing_person["gender"]),
            bigquery.ScalarQueryParameter("height", "FLOAT64", sample_missing_person["height"]),
            bigquery.ScalarQueryParameter("weight", "FLOAT64", sample_missing_person["weight"]),
            bigquery.ScalarQueryParameter("hair_color", "STRING", sample_missing_person["hair_color"]),
            bigquery.ScalarQueryParameter("eye_color", "STRING", sample_missing_person["eye_color"]),
            bigquery.ScalarQueryParameter("distinguishing_marks", "STRING", sample_missing_person["distinguishing_marks"]),
            bigquery.ScalarQueryParameter("clothing_description", "STRING", sample_missing_person["clothing_description"]),
            bigquery.ScalarQueryParameter("last_seen_date", "DATE", sample_missing_person["last_seen_date"]),
            bigquery.ScalarQueryParameter("last_seen_time", "TIME", sample_missing_person["last_seen_time"]),
            bigquery.ScalarQueryParameter("last_seen_address", "STRING", sample_missing_person["last_seen_address"]),
            bigquery.ScalarQueryParameter("last_seen_city", "STRING", sample_missing_person["last_seen_city"]),
            bigquery.ScalarQueryParameter("last_seen_country", "STRING", sample_missing_person["last_seen_country"]),
            bigquery.ScalarQueryParameter("last_seen_postal_code", "STRING", sample_missing_person["last_seen_postal_code"]),
            bigquery.ScalarQueryParameter("last_seen_latitude", "FLOAT64", sample_missing_person["last_seen_latitude"]),
            bigquery.ScalarQueryParameter("last_seen_longitude", "FLOAT64", sample_missing_person["last_seen_longitude"]),
            bigquery.ScalarQueryParameter("circumstances", "STRING", sample_missing_person["circumstances"]),
            bigquery.ScalarQueryParameter("priority", "STRING", sample_missing_person["priority"]),
            bigquery.ScalarQueryParameter("status", "STRING", sample_missing_person["status"]),
            bigquery.ScalarQueryParameter("description", "STRING", sample_missing_person["description"]),
            bigquery.ScalarQueryParameter("medical_conditions", "STRING", sample_missing_person["medical_conditions"]),
            bigquery.ScalarQueryParameter("additional_info", "STRING", sample_missing_person["additional_info"]),
            bigquery.ScalarQueryParameter("photo_url", "STRING", sample_missing_person["photo_url"]),
            bigquery.ScalarQueryParameter("reporter_name", "STRING", sample_missing_person["reporter_name"]),
            bigquery.ScalarQueryParameter("reporter_phone", "STRING", sample_missing_person["reporter_phone"]),
            bigquery.ScalarQueryParameter("reporter_email", "STRING", sample_missing_person["reporter_email"]),
            bigquery.ScalarQueryParameter("relationship", "STRING", sample_missing_person["relationship"]),
        ]
    )

    # Execute the parameterized query
    query_job = client.query(INSERT_MISSING_PERSON_QUERY, job_config=job_config)
    results = query_job.result()  # Wait for the query to complete

    print("✅ Missing person record inserted successfully with AI-generated summary!")
    print(f"🔑 Case ID: {sample_missing_person['id']}")
    print(f"📋 Case Number: {sample_missing_person['case_number']}")
    print(f"👤 Person: {sample_missing_person['name']} {sample_missing_person['surname']}")
    print(f"🔧 Query job completed: {query_job.job_id}")

except Exception as e:
    print(f"❌ Error executing parameterized insert query: {str(e)}")
    print(f"Error type: {type(e).__name__}")

    # Print more detailed error information if available
    if hasattr(e, 'errors') and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")
    if hasattr(e, 'message'):
        print(f"Error message: {e.message}")

Executing parameterized missing person insert with ML summary generation...
✅ Missing person record inserted successfully with AI-generated summary!
🔑 Case ID: a54701be-c15e-47a7-ae1b-3c14c1c5b040
📋 Case Number: CASE-2024-034
👤 Person: John Doe
🔧 Query job completed: 4f762c3f-211c-4576-97ac-1aa36701d6d9


In [23]:
# Query to check the inserted record and view the AI-generated summary
def verify_inserted_record(case_id):
    """
    Query to verify the inserted record using the case ID parameter.

    Args:
        case_id (str): The case ID to verify

    Returns:
        tuple: Query string and job configuration
    """
    job_config = bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("case_id", "STRING", case_id)
        ]
    )

    return get_verify_record_query(), job_config

try:
    print("Querying the inserted record to view AI-generated summary...")

    # Get the query and job config
    check_query, check_job_config = verify_inserted_record(sample_missing_person["id"])

    # Execute the verification query
    query_job = client.query(check_query, job_config=check_job_config)
    results = query_job.result()

    print("✅ Verification query executed successfully!")
    print(f"Total rows returned: {results.total_rows}")

    if results.total_rows > 0:
        print("\n" + "="*80)
        print("INSERTED MISSING PERSON RECORD WITH AI-GENERATED SUMMARY")
        print("="*80)

        for row in results:
            print(f"📋 Case ID: {row.id}")
            print(f"📋 Case Number: {row.case_number}")
            print(f"👤 Name: {row.name} {row.surname}")
            print(f"🎂 Date of Birth: {row.date_of_birth}")
            print(f"⚧ Gender: {row.gender}")
            print(f"📏 Physical: {row.height}cm, {row.weight}kg")
            print(f"👁️ Features: {row.hair_color} hair, {row.eye_color} eyes")
            print(f"🔍 Distinguishing Marks: {row.distinguishing_marks}")
            print(f"👕 Clothing: {row.clothing_description}")
            print(f"📍 Last Seen: {row.last_seen_date} at {row.last_seen_time} in {row.last_seen_city}")
            print(f"⚠️ Circumstances: {row.circumstances}")
            print(f"🏥 Medical Conditions: {row.medical_conditions}")
            print(f"ℹ️ Additional Info: {row.additional_info}")
            print(f"📞 Reporter: {row.reporter_name} ({row.relationship})")
            print(f"🔴 Priority: {row.priority}")
            print(f"📊 Status: {row.status}")
            print(f"📅 Created: {row.created_date}")

            print("\n" + "-"*80)
            print("🤖 AI-GENERATED SUMMARY:")
            print("-"*80)
            print(f"{row.ml_summary}")
            print("-"*80)

            # Calculate summary statistics
            summary_length = len(row.ml_summary) if row.ml_summary else 0
            word_count = len(row.ml_summary.split()) if row.ml_summary else 0
            print("\n📊 Summary Statistics:")
            print(f"   • Length: {summary_length} characters")
            print(f"   • Word count: {word_count} words")
            print(f"   • Format: {'Discursive paragraph' if summary_length > 100 else 'Short summary'}")

    else:
        print(f"⚠️ No records found with case ID '{sample_missing_person['id']}'")

except Exception as e:
    print(f"❌ Error querying inserted record: {str(e)}")
    print(f"Error type: {type(e).__name__}")

    # Print more detailed error information if available
    if hasattr(e, 'errors') and e.errors:
        for error in e.errors:
            print(f"Error details: {error}")

Querying the inserted record to view AI-generated summary...
✅ Verification query executed successfully!
Total rows returned: 1

INSERTED MISSING PERSON RECORD WITH AI-GENERATED SUMMARY
📋 Case ID: a54701be-c15e-47a7-ae1b-3c14c1c5b040
📋 Case Number: CASE-2024-034
👤 Name: John Doe
🎂 Date of Birth: 1990-01-15
⚧ Gender: Male
📏 Physical: 175.0cm, 70.0kg
👁️ Features: Brown hair, Green eyes
🔍 Distinguishing Marks: Tattoo on right arm
👕 Clothing: Black jacket, blue jeans, white sneakers
📍 Last Seen: 2024-08-20 at 18:45:00 in San Francisco
⚠️ Circumstances: Failed to return home after evening jog
🏥 Medical Conditions: Diabetes - requires medication
ℹ️ Additional Info: Regular jogger, knows the area well
📞 Reporter: Jane Doe (Wife)
🔴 Priority: High
📊 Status: Active
📅 Created: 2025-08-22 14:09:17.094578+00:00

--------------------------------------------------------------------------------
🤖 AI-GENERATED SUMMARY:
--------------------------------------------------------------------------------
Joh